In [2]:
### BEGIN Solution (do not delete this comment!)

# >>> your imports here <<<

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
import numpy as np
import time
from tqdm import tqdm
from torchvision.models import resnet34, vgg16

### END Solution (do not delete this comment!)

In [3]:
### BEGIN Solution (do not delete this comment)

train_transform = T.Compose([T.RandomHorizontalFlip(), 
                             T.ToTensor(),
                             T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_transform = T.Compose([T.ToTensor(),
                             T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
train_loader = DataLoader(dataset=train_set, batch_size=128, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = DataLoader(dataset=test_set, batch_size=128, shuffle=False, num_workers=2)

### END Solution (do not delete this comment)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print('Train size', len(train_set))
print('Test size', len(test_set))

Files already downloaded and verified
Files already downloaded and verified
Train size 50000
Test size 10000


In [6]:
def epoch_train(loader, clf, criterion, opt):
    # BEGIN Solution (do not delete this comment!)
    
    final_loss = 0
    final_accuracy = 0
    for image, label in loader:
        image = image.cuda()
        label = label.cuda()
        pred = clf(image)
        loss = criterion(pred, label)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        final_loss += loss.item() * len(image)
        value, max_idx = torch.max(pred, dim=1)
        final_accuracy += torch.sum(max_idx == label)
    final_loss = final_loss / len(loader.dataset)
    final_accuracy = final_accuracy / len(loader.dataset)
    return final_loss, final_accuracy
    # END Solution (do not delete this comment!)

def epoch_test(loader, clf, criterion):
    # BEGIN Solution (do not delete this comment!)
    
    final_loss = 0
    final_accuracy = 0
    for image, label in loader:
        image = image.cuda()
        label = label.cuda()
        pred = clf(image)
        loss = criterion(pred, label)
        
        final_loss += loss.item() * len(image)
        value, max_idx = torch.max(pred, dim=1)
        final_accuracy += torch.sum(max_idx == label)
    final_loss = final_loss / len(loader.dataset)
    final_accuracy = final_accuracy / len(loader.dataset)
    return final_loss, final_accuracy

    # END Solution (do not delete this comment!)

# The function which you are going to use for model training
def train(train_loader, test_loader, clf, criterion, opt, n_epochs=50):
    for epoch in tqdm(range(n_epochs)):
        train_loss, train_acc = epoch_train(train_loader, clf, criterion, opt)
        test_loss, test_acc = epoch_test(test_loader, clf, criterion)

        print(f'[Epoch {epoch + 1}] train loss: {train_loss:.3f}; train acc: {train_acc:.2f}; ' + 
              f'test loss: {test_loss:.3f}; test acc: {test_acc:.2f}')

### Subtask 2.5. PyTorch classifier  ($\geq$83%)
**2.5.1)** After struggling to build the network manually, let us just import a pre-defined neural network architecture from `PyTorch` and train it. Take a look at the available [models](https://pytorch.org/vision/stable/models.html) of `torchvision.models`, pick the one you want and make it achieve $\geq 83$% **test accuracy**. We recommend to consider `VGG` or `ResNet` models. You **should not** use `pretrained=True` -- you job is to train the model from scratch.

An image of ResNet-34, plain and VGG-16 architectures.

![telegram-cloud-photo-size-4-5844643573331503502-w.jpg](attachment:telegram-cloud-photo-size-4-5844643573331503502-w.jpg)

<!-- <img width=750 src="https://miro.medium.com/max/1400/1*6hF97Upuqg_LdsqWY6n_wg.png"> -->

In [8]:
# BEGIN Solution (do not delete this comment!)

# >>> your solution here <<<

clf = vgg16(pretrained=False)
clf.classifier[-1] = nn.Linear(in_features=4096, out_features=10)
clf.cuda()

opt = torch.optim.SGD(clf.parameters(), lr=1e-2, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss()

# END Solution (do not delete this comment!)

In [9]:
train(train_loader, test_loader, clf, criterion, opt, n_epochs=40)

  2%|▎         | 1/40 [00:29<18:54, 29.10s/it]

[Epoch 1] train loss: 2.299; train acc: 0.10; test loss: 2.294; test acc: 0.10


  5%|▌         | 2/40 [00:57<18:03, 28.50s/it]

[Epoch 2] train loss: 2.223; train acc: 0.14; test loss: 2.061; test acc: 0.23


  8%|▊         | 3/40 [01:25<17:29, 28.37s/it]

...

[Epoch 38] train loss: 0.152; train acc: 0.95; test loss: 0.931; test acc: 0.76


 98%|█████████▊| 39/40 [18:27<00:28, 28.42s/it]

[Epoch 39] train loss: 0.138; train acc: 0.95; test loss: 0.906; test acc: 0.77


100%|██████████| 40/40 [18:55<00:00, 28.40s/it]

[Epoch 40] train loss: 0.124; train acc: 0.96; test loss: 1.385; test acc: 0.70
